In [1]:
! pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=b363c098fd7d1f77747ce20b068ed4fe1e9ea7f2e23a57a3bc05946844c0bc60
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.models import load_model
# from keras.layers import LeakyReLU
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
import cv2

In [ ]:
print("[INFO] loading images...")
imagePaths = list(paths.list_images("../input/main-dataset/main dataset"))
# print(imagePaths)
data = []
labels = []

[INFO] loading images...


In [ ]:
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]

	# load the input image (224x224) and preprocess it
    image = load_img(imagePath, target_size=(48, 48))
    image = img_to_array(image)
    if label=='angry':
        l = [1,0,0,0,0,0]
    elif label=='fear':
        l = [0,1,0,0,0,0]
    elif label=='happy':
        l = [0,0,1,0,0,0]
    elif label=='neutral':
        l = [0,0,0,1,0,0]
    elif label=='sad':
        l = [0,0,0,0,1,0]
    elif label=='surprise':
        l = [0,0,0,0,0,1]
        
# 	image = preprocess_input(image)
	# update the data and labels lists, respectively
	# count+=1/
	# if count%10==0:
    data.append(image)
    labels.append(l)
    


In [ ]:
data = np.array(data, dtype="float32")
print(type(data))
labels = np.array(labels)
print("labels",labels.shape)
# perform one-hot encoding on the labels
# lb = LabelEncoder()
# labels = lb.fit_transform(labels1)
# labels = to_categorical(labels)
print(labels,type(labels))

In [ ]:
print(data.shape,labels.shape)
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.2, stratify=labels, random_state=42)
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

In [ ]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer,Conv2D,MaxPool2D,BatchNormalization,ZeroPadding2D,ReLU,SeparableConv2D
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model

import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
# from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow

In [ ]:
def identity_block(X, f, filters, stage, block):
    
    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path 
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1, 1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [ ]:
def convolutional_block(X, f, filters, stage, block, s=2):

    # Defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### 
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet50(input_shape = (72, 72, 3), classes = 7):   
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')
    
    X = Dropout(0.15)(X)

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    
    X = Dropout(0.15)(X)

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')
    
    X = Dropout(0.15)(X)

    

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2,2), padding='same')(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(X)
    X = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(X)
    X = Dropout(0.15)(X)
    X = Dense(156, activation="relu",kernel_regularizer=regularizers.l2(0.001))(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
INIT_LR = 1e-4
EPOCHS = 250
BS = 128


# baseModel = Sequential()
# # baseModel.add(Dense(12, activation='relu', input_tensor = Input(shape=(48,48,3))))
# baseModel.add(InputLayer(input_shape = (48,48,3)))
# # baseModel = tf.keras.applications.Xception(input_shape=(72,72,3),include_top=False,weights='imagenet')
# headModel = baseModel.output
# # headModel = ZeroPadding2D()(headModel)
# headModel = Conv2D(72,3,activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = Conv2D(72,3,activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = SeparableConv2D(72,3,activation="relu",use_bias=True,strides=(2, 2),kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = Conv2D(144,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = SeparableConv2D(144,3,activation="relu",use_bias=True,strides=(1, 1),kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = MaxPool2D(pool_size = (2,2))(headModel)
# headModel = Dropout(0.15)(headModel)

# headModel = Conv2D(288,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Conv2D(288,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# headModel = SeparableConv2D(288,3,activation="relu",use_bias=True,strides=(2, 2),kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = BatchNormalization()(headModel)
# # headModel = MaxPool2D(pool_size = (2,2))(headModel)
# headModel = Dropout(0.15)(headModel)




# headModel = Conv2D(288,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Conv2D(288,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Conv2D(288,3,activation="relu",padding = "same",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = MaxPool2D(pool_size = (2,2))(headModel)


# # construct the head of the model that will be placed on top of the
# # the base model
# # headModel = baseModel.output
# # headModel = AveragePooling2D(pool_size=(1, 1))(headModel)
# headModel = Flatten(name="flatten")(headModel)
# # headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# # headModel = Dropout(0.15)(headModel)
# headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Dense(512, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# headModel = Dropout(0.15)(headModel)
# headModel = Dense(156, activation="relu",kernel_regularizer=regularizers.l2(0.001))(headModel)
# # headModel = Dropout(0.1)(headModel)
# # headModel = Dense(256, activation="relu",kernel_initializer = 'he_normal')(headModel)
# # headModel = Dense(1024, activation="relu")(headModel)
# # headModel = Dense(256, activation="relu")(headModel)
# headModel = Dense(6, activation="softmax")(headModel)

# # place the head FC model on top of the base model (this will become
# # the actual model we will train)
# model = Model(inputs=baseModel.input, outputs=headModel)


# checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
#                              monitor='val_loss',
#                              mode='min',
#                              save_best_only=True,
#                              verbose=0)

# # earlystop = EarlyStopping(monitor='val_loss',
# #                           min_delta=0,
# #                           patience=3,
# #                           verbose=1,
# #                           restore_best_weights=True
# #                           )

# # reduce_lr = ReduceLROnPlateau(monitor='val_loss',
# #                               factor=0.2,
# #                               patience=5,
# #                               verbose=1,
# #                               min_delta=0.0001)

# callbacks = [checkpoint]

# # for layer in baseModel.layers:
# # 	layer.trainable = False

# print(model.summary())

In [ ]:
model = ResNet50(input_shape = (48, 48, 3), classes = 6)

In [ ]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=0)

callbacks = [checkpoint]
print(model.summary())

In [ ]:
# for layer in baseModel.layers:
# 	layer.trainable = False

# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])



In [ ]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
    callbacks=callbacks,
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

model = load_model("EmotionDetectionModel.h5")


# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
print(predIdxs)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
print(predIdxs)

# show a nicely formatted classification report
# print(classification_report(testY.argmax(axis=1), predIdxs,
# 	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("/kaggle/working/model.h5", save_format="h5")



In [ ]:
history_dict = H.history
print(history_dict.keys())

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training & val loss & Accuracy for 25 layer sep-conv network")
plt.xlabel(f"Epoch {EPOCHS}")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/kaggle/working/loss_accuracy_plot")

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
# plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
# plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training & val loss for 55 layer residual conv network")
plt.xlabel(f"Epoch {EPOCHS}")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/kaggle/working/loss_plot")

In [ ]:
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
# plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training & val loss & Accuracy for 25 layer sep-conv network")
plt.xlabel(f"Epoch {EPOCHS}")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("/kaggle/working/accuracy_plot")